### Initial Model Training on Expanded Dataset and Regularisation Experiments
#### File Set-Up

In [ ]:
# solves occasional error with Linux commands and encoding
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
# mount google drive for saving runs and model files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# copy dataset from google drive
!unzip /content/drive/MyDrive/UniStuff/Dissertation/Dataset_zips/WSP_Full_Boxes_Only.zip -d /content/datasets/

Streaming output truncated to the last 5000 lines.
  inflating: /content/datasets/WSP_Full_Boxes_Only/train/labels/2782_jpg.rf.607cfbec890451d1f9fcff8516f4945d_greyscale_reversed.txt  
  inflating: /content/datasets/WSP_Full_Boxes_Only/train/labels/2782_jpg.rf.607cfbec890451d1f9fcff8516f4945d_inc_brightness.txt  
  inflating: /content/datasets/WSP_Full_Boxes_Only/train/labels/2782_jpg.rf.607cfbec890451d1f9fcff8516f4945d_inc_brightness_flipped.txt  
  inflating: /content/datasets/WSP_Full_Boxes_Only/train/labels/2782_jpg.rf.607cfbec890451d1f9fcff8516f4945d_inc_brightness_reversed.txt  
  inflating: /content/datasets/WSP_Full_Boxes_Only/train/labels/2782_jpg.rf.607cfbec890451d1f9fcff8516f4945d_inc_contrast.txt  
  inflating: /content/datasets/WSP_Full_Boxes_Only/train/labels/2782_jpg.rf.607cfbec890451d1f9fcff8516f4945d_inc_contrast_flipped.txt  
  inflating: /content/datasets/WSP_Full_Boxes_Only/train/labels/2782_jpg.rf.607cfbec890451d1f9fcff8516f4945d_inc_contrast_reversed.txt  
  infla

In [ ]:
# install and import ultralytics

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 31.1/78.2 GB disk)


### Model Training
Model trained on expanded dataset for 100 epochs

In [ ]:
# train model on boxes-only dataset, 100 epochs
model_boxes_only = YOLO("yolov8x.pt")

results = model_boxes_only.train(data="/content/datasets/WSP_Full_Boxes_Only/data.yaml", project="/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns", name="model_boxes_only", epochs=100)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=/content/datasets/WSP_Full_Boxes_Only/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_boxes_only, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=T

In [ ]:
# validate initial model
model_boxes_only = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_boxes_only/weights/best.pt")
model_boxes_only.val(plots=True, save_json=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 268 layers, 68132235 parameters, 0 gradients, 257.4 GFLOPs
100%|██████████| 755k/755k [00:00<00:00, 104MB/s]
val: Scanning /content/datasets/WSP_Full_Boxes_Only/valid/labels... 153 images, 1 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<00:00, 1178.92it/s]
val: New cache created: /content/datasets/WSP_Full_Boxes_Only/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descript

### Regularisation Experiments
Model was found to be overfitting to training set, therefore the following regularisation techniques will be experimented with to reduce overfitting.
- increased weight decay
- dropout
- decreased YOLO augmentation
- increased YOLO augmentation

All models included early stopping condition of 10 epochs, so training would stop if validation performance did not improve for 10 consecutive epochs.

In [ ]:
# model with 10x default weight decay
model_increased_weight_decay = YOLO("yolov8x.pt")
results = model_increased_weight_decay.train(data="/content/datasets/WSP_Full_Boxes_Only/data.yaml",
                                             project="/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns",
                                             name="model_increased_weight_decay",
                                             optimizer="Adam",
                                             weight_decay=0.005,
                                             patience=10,
                                             epochs=100,
                                             plots=True)

100%|██████████| 131M/131M [00:00<00:00, 550MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=/content/datasets/WSP_Full_Boxes_Only/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_increased_weight_decay, exist_ok=False, pretrained=False, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False

In [ ]:
# validate increased weight decay model
model_increased_weight_decay = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_increased_weight_decay/weights/best.pt")
model_increased_weight_decay.val(plots=True, save_json=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 268 layers, 68132235 parameters, 0 gradients, 257.4 GFLOPs
100%|██████████| 755k/755k [00:00<00:00, 85.4MB/s]
val: Scanning /content/datasets/WSP_Full_Boxes_Only/valid/labels... 153 images, 1 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<00:00, 1220.03it/s]
val: New cache created: /content/datasets/WSP_Full_Boxes_Only/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descrip

In [ ]:
# save files to google drive
!cp -r /content/runs/ /content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_increased_weight_decay/

In [ ]:
# model using dropout
model_with_dropout = YOLO("yolov8x.pt")
results = model_with_dropout.train(data="/content/datasets/WSP_Full_Boxes_Only/data.yaml",
                                             project="/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns",
                                             name="model_with_dropout",
                                             optimizer="Adam",
                                             dropout=0.05,
                                             patience=10,
                                             epochs=100,
                                             plots=True)

100%|██████████| 131M/131M [00:00<00:00, 451MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=/content/datasets/WSP_Full_Boxes_Only/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_with_dropout, exist_ok=False, pretrained=False, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.05, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, c

In [ ]:
# resume model with dropout training
model_with_dropout = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_with_dropout/weights/last.pt")
results = model_with_dropout.train(resume=True,
                                   optimizer="Adam",
                                   dropout=0.05,
                                   patience=10,
                                   plots=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
yolo/engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_with_dropout/weights/last.pt, data=/content/datasets/WSP_Full_Boxes_Only/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_with_dropout, exist_ok=False, pretrained=False, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.05, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=Fa

In [ ]:
# validate model with dropout
model_with_dropout = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_with_dropout/weights/best.pt")
model_with_dropout.val(plots=True, save_json=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Model summary (fused): 268 layers, 68132235 parameters, 0 gradients, 257.4 GFLOPs
100%|██████████| 755k/755k [00:00<00:00, 14.8MB/s]
val: Scanning /content/datasets/WSP_Full_Boxes_Only/valid/labels... 153 images, 1 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<00:00, 1289.45it/s]
val: New cache created: /content/datasets/WSP_Full_Boxes_Only/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_f

In [ ]:
# model with decreased YOLO augmentation settings
model_augment_disabled = YOLO("yolov8x.pt")
results = model_augment_disabled.train(data="/content/datasets/WSP_Full_Boxes_Only/data.yaml",
                                             project="/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns",
                                             name="model_augment_disabled",
                                       epochs=100,
                                       plots=True,
                                      patience=10,
                                       augment=False,
                                       hsv_h=0.0, hsv_s=0.0,
                                      hsv_v=0.0, degrees=0.0,
                                      translate=0.0, scale=0.0,
                                      shear=0.0, perspective=0.0,
                                      flipud=0.0, fliplr=0.0, mosaic=1,
                                      mixup=0.0, copy_paste=0.0)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=/content/datasets/WSP_Full_Boxes_Only/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_augment_disabled, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, bo

In [ ]:
# validate model with decreased YOLO augmentation
best_augment_disabled_model = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_augment_disabled/weights/best.pt")
best_augment_disabled_model.val(plots=True, save_json=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 268 layers, 68132235 parameters, 0 gradients, 257.4 GFLOPs
val: Scanning /content/datasets/WSP_Full_Boxes_Only/valid/labels.cache... 153 images, 1 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)

In [ ]:
# copy files to google drive
!cp -r /content/runs/detect/val2 /content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_augment_disabled/

In [ ]:
# model with augmentations increased from default
model_augment_increased = YOLO("yolov8x.pt")
results = model_augment_increased.train(data="/content/datasets/WSP_Full_Boxes_Only/data.yaml",
                                             project="/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns",
                                             name="model_augment_increased",
                                       epochs=100,
                                       plots=True,
                                      patience=10,
                                       hsv_h=0.5, hsv_s=0.9,
                                      hsv_v=0.8, degrees=105,
                                      translate=0.6, scale=0.9,
                                      shear=45, perspective=0.0,
                                      flipud=0.5, fliplr=0.5, mosaic=1,
                                      mixup=1.0, copy_paste=0.5)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=/content/datasets/WSP_Full_Boxes_Only/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_augment_increased, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, b

In [ ]:
# continue training model with augmentations increased from default
model_augment_increased = YOLO("/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_augment_increased/weights/last.pt")
results = model_augment_increased.train(resume=True,
                                       plots=True,
                                      patience=10,
                                       hsv_h=0.5, hsv_s=0.9,
                                      hsv_v=0.8, degrees=105,
                                      translate=0.6, scale=0.9,
                                      shear=45, perspective=0.0,
                                      flipud=0.5, fliplr=0.5, mosaic=1,
                                      mixup=1.0, copy_paste=0.5)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
yolo/engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns/model_augment_increased/weights/last.pt, data=/content/datasets/WSP_Full_Boxes_Only/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/drive/MyDrive/UniStuff/Dissertation/ModelRuns, name=model_augment_increased, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, 

In [ ]:
# validate model with increased YOLO augmentation
model_augment_increased.val(plots=True, save_json=True)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 268 layers, 68132235 parameters, 0 gradients, 257.4 GFLOPs
val: Scanning /content/datasets/WSP_Full_Boxes_Only/valid/labels.cache... 153 images, 1 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.